<a href="https://colab.research.google.com/github/SatChittAnand/Seq2Seq-model-newspaper-text-summarization/blob/main/DLWTF_seq2seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
gowrishankarp_newspaper_text_summarization_cnn_dailymail_path = kagglehub.dataset_download('gowrishankarp/newspaper-text-summarization-cnn-dailymail')

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/validation.csv
/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/train.csv
/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/test.csv


In [ ]:
# Load train, validation, and test splits
train_df = pd.read_csv("/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/train.csv")
val_df   = pd.read_csv("/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/validation.csv")
test_df  = pd.read_csv("/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/test.csv")

print("Train shape:", train_df.shape)
print("Validation shape:", val_df.shape)
print("Test shape:", test_df.shape)

# Peek at the first few rows
train_df.head()


Train shape: (287113, 3)
Validation shape: (13368, 3)
Test shape: (11490, 3)


,id,article,highlights
0,0001d1afc246a7964130f43ae940af6bc6c57f01,By . Associated Press . PUBLISHED: . 14:11 EST...,"Bishop John Folda, of North Dakota, is taking ..."
1,0002095e55fcbd3a2f366d9bf92a95433dc305ef,(CNN) -- Ralph Mata was an internal affairs li...,Criminal complaint: Cop used his role to help ...
2,00027e965c8264c35cc1bc55556db388da82b07f,A drunk driver who killed a young woman in a h...,"Craig Eccleston-Todd, 27, had drunk at least t..."
3,0002c17436637c4fe1837c935c04de47adb18e9a,(CNN) -- With a breezy sweep of his pen Presid...,Nina dos Santos says Europe must be ready to a...
4,0003ad6ef0c37534f80b55b4235108024b407f0b,Fleetwood are the only team still to have a 10...,Fleetwood top of League One after 2-0 win at S...


In [ ]:
print(train_df.columns)
print(train_df.iloc[0]["article"][:500])   # preview first 500 chars
print(train_df.iloc[0]["highlights"])


Index(['id', 'article', 'highlights'], dtype='object')
By . Associated Press . PUBLISHED: . 14:11 EST, 25 October 2013 . | . UPDATED: . 15:36 EST, 25 October 2013 . The bishop of the Fargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members in Fargo, Grand Forks and Jamestown to the hepatitis A virus in late September and early October. The state Health Department has issued an advisory of exposure for anyone who attended five churches and took communion. Bishop John Folda (pictured) of the Fargo Catholic Diocese in N
Bishop John Folda, of North Dakota, is taking time off after being diagnosed .
He contracted the infection through contaminated food in Italy .
Church members in Fargo, Grand Forks and Jamestown could have been exposed .


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("facebook/bart-base")

# Example encoding
sample = tokenizer(train_df.iloc[0]["article"], max_length=512, truncation=True, padding="max_length")
summary = tokenizer(train_df.iloc[0]["highlights"], max_length=128, truncation=True, padding="max_length")

print(sample["input_ids"][:20])
print(summary["input_ids"][:20])


config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

[0, 2765, 479, 1562, 977, 479, 29731, 7976, 14849, 1691, 35, 479, 501, 35, 1225, 12936, 6, 564, 779, 1014]
[0, 387, 44517, 610, 41303, 102, 6, 9, 369, 6223, 6, 16, 602, 86, 160, 71, 145, 6443, 479, 50118]


In [ ]:
# import torch
# from torch.utils.data import Dataset, DataLoader

# class CNNDailyMailDataset(Dataset):
#     def __init__(self, df, tokenizer, max_input_len=512, max_output_len=128):
#         self.df = df
#         self.tokenizer = tokenizer
#         self.max_input_len = max_input_len
#         self.max_output_len = max_output_len

#     def __len__(self):
#         return len(self.df)

#     def __getitem__(self, idx):
#         article = self.df.iloc[idx]["article"]
#         summary = self.df.iloc[idx]["highlights"]
#         enc = self.tokenizer(article, max_length=self.max_import torch)
# from torch.utils.data import Dataset, DataLoader

# class CNNDailyMailDataset(Dataset):
#     def __init__(self, df, tokenizer, max_input_len=512, max_output_len=128):
#         self.df = df
#         self.tokenizer = tokenizer
#         self.max_input_len = max_input_len
#         self.max_output_len = max_output_len

#     def __len__(self):
#         return len(self.df)

#     def __getitem__(self, idx):
#         article = self.df.iloc[idx]["article"]
#         summary = self.df.iloc[idx]["highlights"]
#         enc = self.tokenizer(article, max_length=self.max_input_len, truncation=True, padding="max_length", return_tensors="pt")
#         dec = self.tokenizer(summary, max_length=self.max_output_len, truncation=True, padding="max_length", return_tensors="pt")
#         return {
#             "src": enc["input_ids"].squeeze(),
#             "src_mask": enc["attention_mask"].squeeze(),
#             "tgt": dec["input_ids"].squeeze(),
#             "tgt_mask": dec["attention_mask"].squeeze()
#         }

# train_dataset = CNNDailyMatilDataset(train_df, tokenizer)
# train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
# input_len, truncation=True, padding="max_length", return_tensors="pt")
#         dec = self.tokenizer(summary, max_length=self.max_output_len, truncation=True, padding="max_length", return_tensors="pt")
#         return {
#             "src": enc["input_ids"].squeeze(),
#             "src_mask": enc["attention_mask"].squeeze(),
#             "tgt": dec["input_ids"].squeeze(),
#             "tgt_mask": dec["attention_mask"].squeeze()
#         }

# train_dataset = CNNDailyMatilDataset(train_df, tokenizer)
# train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

import torch
from torch.utils.data import Dataset, DataLoader

class CNNDailyMailDataset(Dataset):
    def __init__(self, df, tokenizer, max_input_len=512, max_output_len=128):
        self.df = df
        self.tokenizer = tokenizer
        self.max_input_len = max_input_len
        self.max_output_len = max_output_len

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        article = self.df.iloc[idx]["article"]
        summary = self.df.iloc[idx]["highlights"]

        # Encode article
        enc = self.tokenizer(
            article,
            max_length=self.max_input_len,
            truncation=True,
            padding="max_length",
            return_tensors="pt"
        )

        # Encode summary
        dec = self.tokenizer(
            summary,
            max_length=self.max_output_len,
            truncation=True,
            padding="max_length",
            return_tensors="pt"
        )

        return {
            "src": enc["input_ids"].squeeze(),
            "src_mask": enc["attention_mask"].squeeze(),
            "tgt": dec["input_ids"].squeeze(),
            "tgt_mask": dec["attention_mask"].squeeze()
        }

# ✅ Correct dataset instantiation
train_dataset = CNNDailyMailDataset(train_df, tokenizer)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)


In [ ]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F

class BahdanauAttention(nn.Module):
    def __init__(self, enc_dim, dec_dim, attn_dim):
        super().__init__()
        self.W_h = nn.Linear(enc_dim, attn_dim, bias=False)
        self.W_s = nn.Linear(dec_dim, attn_dim, bias=True)
        self.v = nn.Linear(attn_dim, 1, bias=False)

    def forward(self, dec_state, enc_outputs, mask=None):
        # enc_outputs: [b, T, enc_dim], dec_state: [b, dec_dim]
        Wh = self.W_h(enc_outputs)                      # [b, T, attn_dim]
        Ws = self.W_s(dec_state).unsqueeze(1)          # [b, 1, attn_dim]
        scores = self.v(torch.tanh(Wh + Ws)).squeeze(-1)  # [b, T]
        if mask is not None:
            scores = scores.masked_fill(~mask, -1e9)
        attn_weights = F.softmax(scores, dim=-1)        # [b, T]
        context = torch.bmm(attn_weights.unsqueeze(1), enc_outputs).squeeze(1)  # [b, enc_dim]
        return context, attn_weights

class LuongAttention(nn.Module):
    def __init__(self, enc_dim, dec_dim, general=True):
        super().__init__()
        self.general = general
        self.W = nn.Linear(enc_dim, dec_dim, bias=False) if general else None

    def forward(self, dec_state, enc_outputs, mask=None):
        # dec_state: [b, dec_dim], enc_outputs: [b, T, enc_dim]
        if self.general:
            proj = self.W(enc_outputs)                  # [b, T, dec_dim]
            scores = torch.bmm(proj, dec_state.unsqueeze(-1)).squeeze(-1)  # [b, T]
        else:
            # dot
            scores = torch.bmm(enc_outputs, dec_state.unsqueeze(-1)).squeeze(-1)
        if mask is not None:
            scores = scores.masked_fill(~mask, -1e9)
        attn_weights = F.softmax(scores, dim=-1)        # [b, T]
        context = torch.bmm(attn_weights.unsqueeze(1), enc_outputs).squeeze(1)  # [b, enc_dim]
        return context, attn_weights

class Encoder(nn.Module):
    def __init__(self, vocab_size, emb_dim, hid_dim, num_layers=1, dropout=0.1):
        super().__init__()
        self.emb = nn.Embedding(vocab_size, emb_dim, padding_idx=0)
        self.rnn = nn.LSTM(emb_dim, hid_dim, num_layers=num_layers, bidirectional=True, batch_first=True, dropout=dropout)
        self.dropout = nn.Dropout(dropout)
        self.out_dim = 2 * hid_dim

    def forward(self, x, lengths):
        emb = self.dropout(self.emb(x))
        packed = nn.utils.rnn.pack_padded_sequence(emb, lengths.cpu(), batch_first=True, enforce_sorted=False)
        out_packed, (h, c) = self.rnn(packed)
        outputs, _ = nn.utils.rnn.pad_packed_sequence(out_packed, batch_first=True)
        # concat final states from both directions
        h_cat = torch.cat([h[-2], h[-1]], dim=-1)
        c_cat = torch.cat([c[-2], c[-1]], dim=-1)
        return outputs, (h_cat, c_cat)

class Decoder(nn.Module):
    def __init__(self, vocab_size, emb_dim, enc_dim, dec_hid, attention, num_layers=1, dropout=0.1):
        super().__init__()
        self.emb = nn.Embedding(vocab_size, emb_dim, padding_idx=0)
        self.rnn = nn.LSTM(emb_dim + enc_dim, dec_hid, num_layers=num_layers, batch_first=True, dropout=dropout)
        self.attention = attention
        self.out = nn.Linear(dec_hid + enc_dim, vocab_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, y_prev, dec_state, enc_outputs, enc_mask):
        emb = self.dropout(self.emb(y_prev))  # [b, 1, emb_dim]
        h_t = dec_state[0][-1]                # top layer hidden: [b, dec_hid]
        context, attn = self.attention(h_t, enc_outputs, mask=enc_mask)
        rnn_in = torch.cat([emb, context.unsqueeze(1)], dim=-1)
        out, dec_state = self.rnn(rnn_in, dec_state)
        out = out.squeeze(1)                  # [b, dec_hid]
        logits = self.out(torch.cat([out, context], dim=-1))
        return logits, dec_state, attn

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, bos_id, eos_id, max_len=160):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.bos_id = bos_id
        self.eos_id = eos_id
        self.max_len = max_len

    def forward(self, src, src_lengths, tgt=None, teacher_forcing=0.5):
        enc_outputs, (h0, c0) = self.encoder(src, src_lengths)
        # bridge to decoder hidden size if dimensions differ
        dec_hid = self.decoder.rnn.hidden_size
        if h0.shape[-1] != dec_hid:
            bridge_h = nn.Linear(h0.shape[-1], dec_hid, bias=False).to(h0.device)
            bridge_c = nn.Linear(c0.shape[-1], dec_hid, bias=False).to(c0.device)
            h0 = bridge_h(h0).unsqueeze(0)
            c0 = bridge_c(c0).unsqueeze(0)
        else:
            h0, c0 = h0.unsqueeze(0), c0.unsqueeze(0)

        dec_state = (h0, c0)
        batch_size = src.size(0)
        enc_mask = src.ne(0)  # [b, T]
        outputs = []
        y = torch.full((batch_size, 1), self.bos_id, dtype=torch.long, device=src.device)

        if tgt is not None:
            T = tgt.size(1)
            for t in range(T - 1):
                logits, dec_state, _ = self.decoder(y, dec_state, enc_outputs, enc_mask)
                outputs.append(logits.unsqueeze(1))
                teacher = torch.rand(1).item() < teacher_forcing
                y = (tgt[:, t+1:t+2] if teacher else logits.argmax(-1, keepdim=True))
            return torch.cat(outputs, dim=1)  # [b, T-1, vocab]
        else:
            # greedy decode
            hyps = []
            for t in range(self.max_len):
                logits, dec_state, _ = self.decoder(y, dec_state, enc_outputs, enc_mask)
                y = logits.argmax(-1, keepdim=True)
                hyps.append(y)
                if torch.all(y.squeeze(1) == self.eos_id):
                    break
            return torch.cat(hyps, dim=1)  # [b, <=max_len]


In [ ]:
print(torch.cuda.device_count())   # should print 2
print(torch.cuda.get_device_name(0))
print(torch.cuda.get_device_name(1))


2
Tesla T4
Tesla T4


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

PAD = tokenizer.pad_token_id
BOS = tokenizer.bos_token_id or tokenizer.cls_token_id
EOS = tokenizer.eos_token_id or tokenizer.sep_token_id


In [ ]:
# Hyperparameters
vocab_size = tokenizer.vocab_size
emb_dim = 256
hidden_size = 512

# Choose attention type
attention = BahdanauAttention(enc_dim=hidden_size*2, dec_dim=hidden_size, attn_dim=hidden_size)
# Or: attention = LuongAttention(enc_dim=hidden_size*2, dec_dim=hidden_size, general=True)

# Build encoder and decoder
encoder = Encoder(vocab_size, emb_dim, hidden_size)
decoder = Decoder(vocab_size, emb_dim, encoder.out_dim, hidden_size, attention)


In [ ]:
model = Seq2Seq(encoder, decoder, bos_id=BOS, eos_id=EOS).to(device)


In [ ]:
import torch.optim as optim
import torch.nn as nn


optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss(ignore_index=PAD, label_smoothing=0.1)


In [ ]:
num_epochs = 2  # adjust as needed

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in train_loader:
        src = batch["src"].to(device)
        tgt = batch["tgt"].to(device)
        src_len = batch["src_mask"].sum(dim=1)  # lengths from mask

        logits = model(src, src_len, tgt, teacher_forcing=0.5)
        target = tgt[:, 1:].contiguous()

        loss = criterion(logits.view(-1, logits.size(-1)), target.view(-1))
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {total_loss/len(train_loader):.4f}")
